In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import unidecode
from time import sleep
from tqdm import tqdm

In [2]:
finalUrls = pd.read_csv('url_with_returns.csv', parse_dates = ['date', 'date2','filingDate', 'reportDate', 'reportDate2'])
finalUrls = finalUrls[['date', 'date2','permno', 'comnam', 'ncusip', 'ticker', 'gvkey','cik', 'filingDate', 'reportDate', 
                       'reportDate2','form', 'size', 'isXBRL', 'isInlineXBRL', 'url', 'url_txt']]
finalUrls = finalUrls[finalUrls['reportDate'] >= '2017-01-01'].reset_index(drop = True)
print(finalUrls.dtypes)
print(finalUrls.shape)
finalUrls.head()

date            datetime64[ns]
date2           datetime64[ns]
permno                   int64
comnam                  object
ncusip                  object
ticker                  object
gvkey                    int64
cik                    float64
filingDate      datetime64[ns]
reportDate      datetime64[ns]
reportDate2     datetime64[ns]
form                    object
size                   float64
isXBRL                   int64
isInlineXBRL             int64
url                     object
url_txt                 object
dtype: object
(9436, 17)


C:\Users\JuliJaramillo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date,date2,permno,comnam,ncusip,ticker,gvkey,cik,filingDate,reportDate,reportDate2,form,size,isXBRL,isInlineXBRL,url,url_txt
0,2017-03-31,2017-03-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2017-05-10,2017-03-31,2017-03-01,10-Q,5389784.0,1,0,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
1,2017-06-30,2017-06-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2017-08-09,2017-06-30,2017-06-01,10-Q,5750164.0,1,0,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
2,2017-09-30,2017-09-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2017-11-08,2017-09-30,2017-09-01,10-Q,5940459.0,1,0,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
3,2017-12-31,2017-12-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2018-03-01,2017-12-31,2017-12-01,10-K,12265797.0,1,0,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
4,2018-03-31,2018-03-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2018-05-10,2018-03-31,2018-03-01,10-Q,5312653.0,1,0,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...


In [3]:
finalUrls[(finalUrls.isInlineXBRL == 1) & ((finalUrls.isXBRL == 1))] # lots of spaces, dates, etc.
#finalUrls[(finalUrls.isInlineXBRL == 1) & ((finalUrls.isXBRL == 0))] #nothing
#finalUrls[(finalUrls.isInlineXBRL == 0) & ((finalUrls.isXBRL == 1))] #we are good here
#finalUrls[(finalUrls.isInlineXBRL == 0) & ((finalUrls.isXBRL == 0))] #we are good

,date,date2,permno,comnam,ncusip,ticker,gvkey,cik,filingDate,reportDate,reportDate2,form,size,isXBRL,isInlineXBRL,url,url_txt
9,2019-06-30,2019-06-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2019-08-02,2019-06-30,2019-06-01,10-Q,13648560.0,1,1,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
10,2019-09-30,2019-09-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2019-11-04,2019-09-30,2019-09-01,10-Q,15635801.0,1,1,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
11,2019-12-31,2019-12-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2020-03-02,2019-12-31,2019-12-01,10-K,23975396.0,1,1,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
12,2020-03-31,2020-03-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2020-05-01,2020-03-31,2020-03-01,10-Q,12722353.0,1,1,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
13,2020-06-30,2020-06-01,60986,NEWELL BRANDS INC,65122910,NWL,7875,814453.0,2020-08-05,2020-06-30,2020-06-01,10-Q,15051393.0,1,1,https://www.sec.gov/Archives/edgar/data/000081...,https://www.sec.gov/Archives/edgar/data/814453...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9431,2021-09-30,2021-09-01,18312,MODERNA INC,60770K10,MRNA,34410,1682852.0,2021-11-04,2021-09-30,2021-09-01,10-Q,11653425.0,1,1,https://www.sec.gov/Archives/edgar/data/000168...,https://www.sec.gov/Archives/edgar/data/168285...
9432,2021-09-30,2021-09-01,75694,BIO TECHNE CORP,09073M10,TECH,15414,842023.0,2021-11-08,2021-09-30,2021-09-01,10-Q,6578134.0,1,1,https://www.sec.gov/Archives/edgar/data/000084...,https://www.sec.gov/Archives/edgar/data/842023...
9433,2021-09-30,2021-09-01,15850,MATCH GROUP INC NEW,57667L10,MTCH,26061,891103.0,2021-11-08,2021-09-30,2021-09-01,10-Q,11005811.0,1,1,https://www.sec.gov/Archives/edgar/data/000089...,https://www.sec.gov/Archives/edgar/data/891103...
9434,2021-09-30,2021-09-01,17700,CERIDIAN H C M HOLDING INC,15677J10,CDAY,23546,1725057.0,2021-11-03,2021-09-30,2021-09-01,10-Q,16025097.0,1,1,https://www.sec.gov/Archives/edgar/data/000172...,https://www.sec.gov/Archives/edgar/data/172505...


In [4]:
#Define user agent and headers
user_agent = 'jdjp_97@hotmail.com'
headers = {'User-Agent':user_agent} 

#Get the url
directory = './Forms/'
directoryRaw = './RawForms/'

for idx in tqdm(range(8786, finalUrls.shape[0])):
    #print(url)
    url = finalUrls.iloc[idx, -2]
    filingDate = str(finalUrls.iloc[idx, 8]).split(' ')[0]
    isInlineXBRL = finalUrls.iloc[idx, -3]
    cik = str(int(finalUrls.iloc[idx, 7])).zfill(10)
    
    strToSave = directory + cik + '_' + filingDate + '.txt'
    strToSaveRaw = directoryRaw + cik + '_' + filingDate + '.txt'

    #Do the request and retrieve the text
    request = requests.get(url, headers = headers)
    form = request.text 
    
    with open(strToSaveRaw, 'w', encoding="utf-8") as file:
        file.write(form)
   
    soup = BeautifulSoup(form, 'html.parser')
    
    #inlineXBRL files, we delete the first div (XBRL format)
    if (isInlineXBRL == 1):
        toExtract = soup.find('div', style = 'display:none')
        if toExtract is not None:
            toExtract.extract()
            
     #Remove every table with more than 25% of number characters
    
    for table in soup.find_all("table"):
        textTable = unidecode.unidecode( table.get_text() )
        num = len(''.join(re.findall(r'[0-9]', textTable)))
        den = len(textTable) - len(''.join(re.findall(' +', textTable)))
        if den == 0:
            table.extract()
        else:
            prop = num/den
            if prop > 0.25:
                table.extract()
        
    #decoding (\xa0)
    form = unidecode.unidecode( soup.get_text() ).strip().lower()
    
    #Remove numbers
    form_no_numbers = re.sub(r'\d', ' ', form)
    
    #Remove the words table of contents and htm
    form_no_numbers = form_no_numbers.replace('table of contents', ' ')
    form_no_numbers = form_no_numbers.replace('htm', ' ')
    
    #Remove punctuation and replace for a single space
    form_no_punctuation = re.sub(r'[^\w\s]', ' ', form_no_numbers)
    
    #Remove words with one character
    form_no_single_char = re.sub(r"\b[a-zA-Z]\b", " ", form_no_punctuation)
    
    #Remove when there is more than one space
    form_no_double_spaces = re.sub(r"[\n]+", " ", form_no_single_char)
    form_no_double_spaces = re.sub(' +', ' ', form_no_double_spaces).strip()

    with open(strToSave, 'w', encoding="utf-8") as file:
        file.write(form_no_double_spaces)

    if (idx % 100 == 0) & (idx != 0):
        print(idx)


  2%|█▊                                                                               | 15/650 [00:45<13:58,  1.32s/it]

8800


 18%|██████████████▏                                                                 | 115/650 [03:28<08:12,  1.09it/s]

8900


 33%|██████████████████████████▍                                                     | 215/650 [06:26<19:08,  2.64s/it]

...

 79%|███████████████████████████████████████████████████████████████▍                | 515/650 [14:05<03:07,  1.39s/it]

9300


 95%|███████████████████████████████████████████████████████████████████████████▋    | 615/650 [15:58<00:49,  1.42s/it]

9400


100%|████████████████████████████████████████████████████████████████████████████████| 650/650 [16:39<00:00,  1.54s/it]


In [5]:
soup.find('div', style = 'display:none')